# Analysis Template: Michaelis-Menten Kinetics
Updated 5/7/24 DFM

In [ ]:
#enables autoreloding of modules
%load_ext autoreload
%autoreload 2

from htbam_db_api.htbam_db_api import LocalHtbamDBAPI
from htbam_analysis.analysis.experiment import HTBAMExperiment

#enable inline plotting of matplotlib figures
%matplotlib inline

#set the figure format to SVG
%config InlineBackend.figure_format = 'svg'

## 1. Connect DB Api

In [ ]:
### PARAMETERS:
EGFP_SLOPE = 91900.03
EGFP_SLOPE_CONC_UNITS = 'nM' #RFU/nM

root = '/Users/duncanmuir/Downloads/20240422-MM-Mpro/'
db_conn = LocalHtbamDBAPI(standard_curve_data_path= root + 'd1_2_StandardSeries_Analysis.csv', standard_name="IDK", standard_type="IDK", standard_units="uM",
                         kinetic_data_path= root+ 'd1_TitrationSeries_Analysis.csv', kinetic_name="IDK", kinetic_type="IDK", kinetic_units="uM")

htbam_experiment = HTBAMExperiment(db_conn)

## 2. Standards

In [ ]:
htbam_experiment.fit_standard_curve('standard_0')

In [ ]:
htbam_experiment.plot_standard_curve_chip('standard_0')

## 3. Fit Initial Rates

In [ ]:
htbam_experiment.fit_initial_rates('kinetics_0', 
                                   'standard_0',
                                   max_rxn_perc=40,
                                   starting_timepoint_index=1,
                                   max_rxn_time=150,
                                   #substrate_conc=60 # Only specified if you have a fixed substrate concentration
                                   )



In [ ]:
htbam_experiment.plot_initial_rates_chip('kinetics_0', 
    time_to_plot=.5, 
    subtract_zeroth_point=True # this doesn't fully work, but can help for visualization
    )

## 4. Filter initial rates

In [ ]:
htbam_experiment.compute_enzyme_concentration('kinetics_0', EGFP_SLOPE)

In [ ]:
htbam_experiment.filter_initial_rates('kinetics_0',
                                      'standard_0',
                                      standard_curve_r2_cutoff = 0.98,
                                      expression_threshold = 1.0,
                                      initial_rate_R2_threshold = 0.5, 
                                      positive_initial_slope_filter = True)

In [ ]:
htbam_experiment.plot_filtered_initial_rates_chip('kinetics_0')

## 5. Fit Michaelis-Menten:

In [ ]:
substrate_conc_unit = htbam_experiment.get_concentration_units('standard_0')
if  substrate_conc_unit != EGFP_SLOPE_CONC_UNITS: print('Substrate concentration units do not match EGFP standard curve units! \n{} != {}'.format(substrate_conc_unit, EGFP_SLOPE_CONC_UNITS))

unit_converstion = 0.001 #convert FROM eGFP units TO substrate units (in this case, nM to uM)


In [ ]:
htbam_experiment.fit_mm('kinetics_0', enzyme_conc_conversion=unit_converstion)

In [ ]:
htbam_experiment.filter_mm('kinetics_0',
                            z_score_threshold_mm = 1.5,
                            z_score_threshold_expression = 1.5,
                            save_intermediate_data = False)

In [ ]:
htbam_experiment.plot_filtered_mm('kinetics_0', 
                                  enzyme_conc_conversion=unit_converstion,
                                  show_average_fit=True)

## 6. Export to CSV

In [ ]:
htbam_experiment.export_mm_result_csv(path_to_save=root,
                                        run_name='kinetics_0')